In [ ]:
!pip install '/content/pyGPB-0.0.1.tar.gz'

Processing ./pyGPB-0.0.1.tar.gz
  Created wheel for pyGPB: filename=pyGPB-0.0.1-py3-none-any.whl size=11720 sha256=4c42a01629580a21453ae74430948560b090283b592fc1296ad8e411e982cb4c
  Stored in directory: /root/.cache/pip/wheels/18/9e/73/f0be66f2d221d99a8ecad80a5f64821626f2a6c13e8f039eab
Successfully built pyGPB


In [ ]:
import numpy as np
import pickle
import timeit
import numba
import math
import cupy as cp
from pyGPB._cpu_methods import _GPB_Naive, _GPB_CF, _GPB_DP,  _GPB_DC_FFT, _GPB_DP_DC_FFT_Combo
from pyGPB._gpu_methods import _GPB_DP_CUDA, _GPB_DP_DC_FFT_CUDA
from cupyx.time import repeat
from matplotlib import pyplot as plt

In [ ]:
# Test space definition
NUMBER_OF_TEST_CASES_PER_SCENARIO = 16
M_space = np.logspace(1,4,num=25, dtype=np.int32)
N_space = np.logspace(3,7,num=25, dtype=np.int32)
N_space_small = np.array([100,200,500,1000,2000,5000,10000])
M_space_small = np.array([5,10,15,20,25])
weight_distribution_space = ['uniform', 'poisson', 'geometric']
prob_distribution_space = ['uniform', 'near_0', 'near_1']
N_space, M_space, N_space_small, M_space_small, weight_distribution_space, prob_distribution_space

(array([    1000,     1467,     2154,     3162,     4641,     6812,
           10000,    14677,    21544,    31622,    46415,    68129,
          100000,   146779,   215443,   316227,   464158,   681292,
         1000000,  1467799,  2154434,  3162277,  4641588,  6812920,
        10000000], dtype=int32),
 array([   10,    13,    17,    23,    31,    42,    56,    74,   100,
          133,   177,   237,   316,   421,   562,   749,  1000,  1333,
         1778,  2371,  3162,  4216,  5623,  7498, 10000], dtype=int32),
 array([  100,   200,   500,  1000,  2000,  5000, 10000]),
 array([ 5, 10, 15, 20, 25]),
 ['uniform', 'poisson', 'geometric'],
 ['uniform', 'near_0', 'near_1'])

In [ ]:
# Load test data
test_case_weights = pickle.load( open( '/content/drive/MyDrive/MSc Project/test_cases_weights.p', "rb" ) )
test_case_probs = pickle.load( open( '/content/drive/MyDrive/MSc Project/test_cases_probabilities.p', "rb" ) )

In [ ]:
def run_for_cases(function, probs_cases, weights_cases):
    for probs, weights in zip(probs_cases, weights_cases):
        function(probs, weights)

In [ ]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               63
Model name:          Intel(R) Xeon(R) CPU @ 2.30GHz
Stepping:            0
CPU MHz:             2299.998
BogoMIPS:            4599.99
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            46080K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs 

In [ ]:
!free -h --si | awk  '/Mem:/{print $2}'

13G


In [ ]:
!nvidia-smi

Sun Aug 22 07:15:12 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
cp.cuda.Device().attributes

{'AsyncEngineCount': 2,
 'CanFlushRemoteWrites': 0,
 'CanMapHostMemory': 1,
 'CanUseHostPointerForRegisteredMem': 1,
 'ClockRate': 1328500,
 'ComputeMode': 0,
 'ComputePreemptionSupported': 1,
 'ConcurrentKernels': 1,
 'ConcurrentManagedAccess': 1,
 'CooperativeLaunch': 1,
 'CooperativeMultiDeviceLaunch': 1,
 'DirectManagedMemAccessFromHost': 0,
 'EccEnabled': 1,
 'GlobalL1CacheSupported': 1,
 'GlobalMemoryBusWidth': 4096,
 'GpuOverlap': 1,
 'HostNativeAtomicSupported': 0,
 'HostRegisterReadOnlySupported': 1,
 'HostRegisterSupported': 1,
 'Integrated': 0,
 'IsMultiGpuBoard': 0,
 'KernelExecTimeout': 0,
 'L2CacheSize': 4194304,
 'LocalL1CacheSupported': 1,
 'ManagedMemory': 1,
 'MaxBlockDimX': 1024,
 'MaxBlockDimY': 1024,
 'MaxBlockDimZ': 64,
 'MaxBlocksPerMultiprocessor': 32,
 'MaxGridDimX': 2147483647,
 'MaxGridDimY': 65535,
 'MaxGridDimZ': 65535,
 'MaxPitch': 2147483647,
 'MaxRegistersPerBlock': 65536,
 'MaxRegistersPerMultiprocessor': 65536,
 'MaxSharedMemoryPerBlock': 49152,
 'MaxS

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!nvidia-smi -L

In [ ]:
weight_distribution = 'uniform'
prob_distribution = 'uniform'
for N in N_space:
    for M in M_space:
        if N > M:
            times = dict()
            for func_name, func in {'_GPB_DP_CUDA':_GPB_DP_CUDA, '_GPB_DP_DC_FFT_CUDA':_GPB_DP_DC_FFT_CUDA}.items():
                results = repeat(func=run_for_cases, args=(func, test_case_probs[(prob_distribution, M)], test_case_weights[(weight_distribution, N, M)]), n_repeat=16, n_warmup=4)
                best_time = (results.cpu_times+results.gpu_times).min() / 16
                times[func_name] = best_time
            for func in ['_GPB_DP',  '_GPB_DC_FFT', '_GPB_DP_DC_FFT_Combo']:
                initial_time_estimate = timeit.timeit(f"{func}(test_case_probs[('{prob_distribution}', {M})][0], test_case_weights[('{weight_distribution}', {N}, {M})][0])", globals=globals(), number=1)
                if initial_time_estimate > 1.0:
                    best_time = initial_time_estimate
                else:
                    approx_runs_in_1_second = 1 / initial_time_estimate
                    if approx_runs_in_1_second < NUMBER_OF_TEST_CASES_PER_SCENARIO**2:
                        cases_to_use = int(math.sqrt(approx_runs_in_1_second))
                    else:
                        cases_to_use = NUMBER_OF_TEST_CASES_PER_SCENARIO
                    repeats = max(int(approx_runs_in_1_second // cases_to_use),1)
                    statement = f"""for probs, weights in zip(probs_cases, weights_cases): {func}(probs, weights)
                    """
                    setup = f"""probs_cases = test_case_probs[('{prob_distribution}', {M})][:{cases_to_use}]\nweights_cases = test_case_weights[('{weight_distribution}', {N}, {M})][:{cases_to_use}]
                    """
                    timer = timeit.Timer(statement, setup=setup, globals=globals())
                    results = timer.repeat(repeat=repeats, number=1)
                    best_time = min(results)/cases_to_use
                times[func] = best_time
            print(N, M, times['_GPB_DP'], times['_GPB_DC_FFT'], times['_GPB_DP_DC_FFT_Combo'], times['_GPB_DP_CUDA'], times['_GPB_DP_DC_FFT_CUDA'])

1000 10 8.109312503279398e-06 0.00012903837500033433 8.326249997026025e-06 0.00021992831532776334 0.0023435131776543016
1000 13 1.0140625001042736e-05 0.00015673531250115502 1.0292062498251653e-05 0.0002224744347074079 0.002590290735731287
1000 17 1.2570437498027331e-05 0.0001920326875008982 1.2773000001686796e-05 0.00022577268635376414 0.0028999702777741734
1000 23 1.6359562501122582e-05 0.0002462966875000916 1.6553499996518894e-05 0.00022538368577176016 0.00322490257277633
1000 31 2.1893187501120792e-05 0.0003094626250046417 2.2141562496358347e-05 0.00023235343535355922 0.0037947678536827427
1000 42 2.8921749994026413e-05 0.0003956638124975598 2.9989812503572466e-05 0.00023598894111745493 0.004154632966947531
1000 56 3.946506249974391e-05 0.000500393062502269 3.8621812500139185e-05 0.00027430618174747675 0.004597418887418655
1000 74 5.21801249959708e-05 0.0006322671874983143 5.2479750003442405e-05 0.0003213472504172614 0.005222636946354271
1000 100 7.061112499684441e-05 0.00081574256

In [ ]:
weight_distribution = 'uniform'
prob_distribution = 'uniform'
for N in [1000,10000,100000,1000000,10000000]:
    for M in M_space:
        if N > M:
            times = dict()
            for func in ['_GPB_CF']:
                initial_time_estimate = timeit.timeit(f"{func}(test_case_probs[('{prob_distribution}', {M})][0], test_case_weights[('{weight_distribution}', {N}, {M})][0])", globals=globals(), number=1)
                if initial_time_estimate > 1.0:
                    best_time = initial_time_estimate
                else:
                    approx_runs_in_1_second = 1 / initial_time_estimate
                    if approx_runs_in_1_second < NUMBER_OF_TEST_CASES_PER_SCENARIO**2:
                        cases_to_use = int(math.sqrt(approx_runs_in_1_second))
                    else:
                        cases_to_use = NUMBER_OF_TEST_CASES_PER_SCENARIO
                    repeats = max(int(approx_runs_in_1_second // cases_to_use),1)
                    statement = f"""for probs, weights in zip(probs_cases, weights_cases): {func}(probs, weights)
                    """
                    setup = f"""probs_cases = test_case_probs[('{prob_distribution}', {M})][:{cases_to_use}]\nweights_cases = test_case_weights[('{weight_distribution}', {N}, {M})][:{cases_to_use}]
                    """
                    timer = timeit.Timer(statement, setup=setup, globals=globals())
                    results = timer.repeat(repeat=repeats, number=1)
                    best_time = min(results)/cases_to_use
                times[func] = best_time
            print(N, M, times['_GPB_CF'])

1000 10 0.00039839499913796317
1000 13 0.0004323745624787989
1000 17 0.0005469110000149158
1000 23 0.0007320878124801311
1000 31 0.0009751635000156966
1000 42 0.0013066054375485692
1000 56 0.0017159937499400257
1000 74 0.002235000437508461
1000 100 0.0029618925624390613
1000 133 0.003841498923205878
1000 177 0.0049401256666593936
1000 237 0.006403597363631971
1000 316 0.008240603199919861
1000 421 0.010733040888832571
1000 562 0.013928793000104633
1000 749 0.017444793142983275
10000 10 0.0035561377143104827
10000 13 0.004431111571518288
10000 17 0.00567622508333443
10000 23 0.007415642500018293
10000 31 0.009921266111228356
10000 42 0.01316864299997178
10000 56 0.017519702857108705
10000 74 0.022905582666680857
10000 100 0.03031367420007882
10000 133 0.040009386249948875
10000 177 0.052534027000092465
10000 237 0.0695337873336636
10000 316 0.09303794699978123
10000 421 0.1226333690001411
10000 562 0.16379560849964037
10000 749 0.21740168549968075
10000 1000 0.28949802500028454
10000 13